# Project 2 - Ames Housing Data and Kaggle Challenge 
## Notebook 2/4: Cleaning and Modfications of Test Data

#### Kristina Joos
 
Notebook 1: Cleaning and Modfications of Train Data.  
Notebook 2: Cleaning and Modfications of Test Data.  
Notebook 3: Creation of the Null Treatment Data Frame.  
Notebook 4: Modeling.  

---

## 2. EDA and Cleaning

### 2.1 Importing libraries
---

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

%matplotlib inline

### 2.2 Load the Data
---

In [119]:
test_df = pd.read_csv("../data/test.csv")        

### 2.3 Data Cleaning
___

#### 2.3.1  Overview and optimizing column names

In [120]:
test_df.shape

(879, 80)

In [121]:
test_df.head(2)

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD


In [122]:
# Renaming Columns to snake case.
test_df.rename(columns = lambda x: x.lower().replace(' ','_'), inplace = True)

__Fill NaN with None:__

In [123]:
# select all rows that have 'Fill NaN with None' as tratment from nulltrain_df and save as fill_in_none Data Frame.

fill_in_none = nulltrain_df.loc[nulltrain_df['treatment']== 'Fill NaN with None',:]

In [124]:
# Peplace np.nan with 'none' in train_df for all columns names that are indices in fill_in_none:

test_df[fill_in_none.index]=test_df[fill_in_none.index].replace(np.nan, 'none')

In [125]:
# Column 'mas_vnr_type' contains capitalized None. Changing it to NaN:

test_df['mas_vnr_type']=test_df['mas_vnr_type'].replace('None', 'none')

In [126]:
# Checking if all the null values that should be filled with none are filled:

test_df[fill_in_none.index].isnull().sum()

pool_qc           0
misc_feature      0
alley             0
fence             0
fireplace_qu      0
garage_yr_blt     0
garage_cond       0
garage_qual       0
garage_finish     0
garage_type       0
bsmt_exposure     0
bsmtfin_type_2    0
bsmt_cond         0
bsmt_qual         0
bsmtfin_type_1    0
mas_vnr_type      0
dtype: int64

__Fill NaN with 0.0:__

In [127]:
# select all rows that have 'Fill with float 0,0' as tratment from nulltrain_df and save as fill_in_zero Data Frame.

fill_in_zero = nulltrain_df.loc[nulltrain_df['treatment']== 'Fill with float 0,0',:]

In [128]:
# Peplace np.nan with 0.0 in train_df for all columns names that are indices in fill_in_zero:

test_df[fill_in_zero.index]=test_df[fill_in_zero.index].replace(np.nan, 0.0)

In [129]:
# Checking if all the null values that should be filled with 0.0 are filled:

test_df[fill_in_zero.index].isnull().sum()

mas_vnr_area      0
bsmt_half_bath    0
bsmt_full_bath    0
garage_cars       0
bsmtfin_sf_1      0
bsmtfin_sf_2      0
bsmt_unf_sf       0
garage_area       0
total_bsmt_sf     0
dtype: int64

__Fill NaN in lot_frontage with median value of the lot_frontage in the neighborhood:__ 

In [130]:
test_df['lot_frontage'] = test_df.groupby('neighborhood')['lot_frontage'].transform(lambda x: x.fillna(x.median()) )

In [131]:
test_df['lot_frontage']=test_df['lot_frontage'].transform(lambda x: x.fillna(x.mean()) )

In [132]:
test_df['lot_frontage'].isnull().sum()

0


#### 2.3.2 Transforming Categorical Data to Numerical Data

In [133]:
# Select and safe all columns that contain string (object) data.

object_columns = test_df.select_dtypes(include=['object']).columns

In [134]:
# Transform ordered categorical string data into numberical string data.
# Make dictionary that has categorical string as key and int as value.

con_to_number = {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, np.nan: 0, 'none': 0}



In [135]:
# Replace categotical string data with int from con_to_number dictionary.

test_df[['pool_qc',
          'bsmt_cond',
          'bsmt_qual',
          'heating_qc',
          'fireplace_qu',
          'kitchen_qual',
          'garage_qual',
          'garage_cond',
          'exter_cond',
          'exter_qual']] = test_df[['pool_qc',
                                     'bsmt_cond',
                                     'bsmt_qual',
                                     'heating_qc',
                                     'fireplace_qu',
                                     'kitchen_qual',
                                     'garage_qual',
                                     'garage_cond',
                                     'exter_cond', 
                                     'exter_qual']].replace(con_to_number);


In [136]:
# Change dtype from float to int.

test_df[['bsmt_cond',
          'bsmt_qual',
          'heating_qc',
          'fireplace_qu',
          'pool_qc',
          'kitchen_qual',
          'garage_qual',
          'garage_cond',
          'exter_cond',
          'exter_qual']] = test_df[['bsmt_cond',
                                     'bsmt_qual',
                                     'heating_qc',
                                     'fireplace_qu',
                                     'pool_qc',
                                     'kitchen_qual',
                                     'garage_qual',
                                     'garage_cond',
                                     'exter_cond', 
                                     'exter_qual']].astype('int')

In [137]:
# Checking dtypes.

test_df[['bsmt_cond',
          'bsmt_qual',
          'heating_qc',
          'fireplace_qu',
          'pool_qc',
          'kitchen_qual',
          'garage_qual',
          'garage_cond',
          'exter_cond',
          'exter_qual']].dtypes


bsmt_cond       int64
bsmt_qual       int64
heating_qc      int64
fireplace_qu    int64
pool_qc         int64
kitchen_qual    int64
garage_qual     int64
garage_cond     int64
exter_cond      int64
exter_qual      int64
dtype: object

__Make dummies for remaining object columns:__

In [138]:
object_columns

Index(['ms_zoning', 'street', 'alley', 'lot_shape', 'land_contour',
       'utilities', 'lot_config', 'land_slope', 'neighborhood', 'condition_1',
       'condition_2', 'bldg_type', 'house_style', 'roof_style', 'roof_matl',
       'exterior_1st', 'exterior_2nd', 'mas_vnr_type', 'exter_qual',
       'exter_cond', 'foundation', 'bsmt_qual', 'bsmt_cond', 'bsmt_exposure',
       'bsmtfin_type_1', 'bsmtfin_type_2', 'heating', 'heating_qc',
       'central_air', 'electrical', 'kitchen_qual', 'functional',
       'fireplace_qu', 'garage_type', 'garage_yr_blt', 'garage_finish',
       'garage_qual', 'garage_cond', 'paved_drive', 'pool_qc', 'fence',
       'misc_feature', 'sale_type'],
      dtype='object')

In [139]:
# I choose a couple of features I would like to use in my model and turn them into dummies:

dummie_list = ["neighborhood",'bldg_type','house_style','heating','central_air','garage_type','utilities']

In [140]:
# Make dummie Data Frame:

dummie_df = pd.get_dummies(test_df[dummie_list], drop_first=True)

__Concat train_df with dummie_list_df:__

In [141]:
test_df_dummies = pd.concat([test_df, dummie_df], axis = 1)

In [142]:
test_df_dummies.head(2)

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,heating_GasW,heating_Grav,central_air_Y,garage_type_Attchd,garage_type_Basment,garage_type_BuiltIn,garage_type_CarPort,garage_type_Detchd,garage_type_none,utilities_NoSewr
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,0,0,0,0,1,0,0
1,2718,905108090,90,RL,75.0,9662,Pave,none,IR1,Lvl,...,0,0,1,1,0,0,0,0,0,0


__Combine correlated features/ Making interaction terms:__

In [143]:
test_df.columns

Index(['id', 'pid', 'ms_subclass', 'ms_zoning', 'lot_frontage', 'lot_area',
       'street', 'alley', 'lot_shape', 'land_contour', 'utilities',
       'lot_config', 'land_slope', 'neighborhood', 'condition_1',
       'condition_2', 'bldg_type', 'house_style', 'overall_qual',
       'overall_cond', 'year_built', 'year_remod/add', 'roof_style',
       'roof_matl', 'exterior_1st', 'exterior_2nd', 'mas_vnr_type',
       'mas_vnr_area', 'exter_qual', 'exter_cond', 'foundation', 'bsmt_qual',
       'bsmt_cond', 'bsmt_exposure', 'bsmtfin_type_1', 'bsmtfin_sf_1',
       'bsmtfin_type_2', 'bsmtfin_sf_2', 'bsmt_unf_sf', 'total_bsmt_sf',
       'heating', 'heating_qc', 'central_air', 'electrical', '1st_flr_sf',
       '2nd_flr_sf', 'low_qual_fin_sf', 'gr_liv_area', 'bsmt_full_bath',
       'bsmt_half_bath', 'full_bath', 'half_bath', 'bedroom_abvgr',
       'kitchen_abvgr', 'kitchen_qual', 'totrms_abvgrd', 'functional',
       'fireplaces', 'fireplace_qu', 'garage_type', 'garage_yr_blt',
       'g

In [144]:
# Make "total_liv_sqft" column out of "gr_liv_area", "1st_flr_sf", 
# "2nd_flr_sfand", "total_bsmt_sf", "bsmtfin_sf_1".

test_df_dummies['total_liv_sqft'] = \
test_df_dummies['gr_liv_area']+test_df_dummies['1st_flr_sf']+test_df_dummies['2nd_flr_sf']\
+test_df_dummies['total_bsmt_sf']+test_df_dummies['bsmtfin_sf_1']



In [145]:
test_df_dummies['total_bsmt_1st_flr_sf']=test_df_dummies['total_bsmt_sf']*test_df_dummies['1st_flr_sf']

In [146]:
# Make "garage_total" column out of "garage_area" and "garage_cars".

test_df_dummies['garage_total'] = test_df_dummies['garage_area']*test_df_dummies['garage_cars']



### 2.4 Saving Cleaned and Modified Train Data
___

In [147]:
test_df_dummies.to_csv(r"../data/test_df_dummies.csv")

### 2.5 Add log columns for overall_qual and total_liv_sqft for Model 2
___

In [148]:
test_df_dummies_model_2 =test_df_dummies

In [149]:
test_df_dummies_model_2['overall_qual_log']=test_df_dummies_model_2['overall_qual'].map(np.log)
test_df_dummies_model_2['total_liv_sqft_log']=test_df_dummies_model_2['total_liv_sqft'].map(np.log)

In [150]:
test_df_dummies_model_2.to_csv(r"../data/test_df_dummies_model_2.csv")

### 2.6 Adding Polynominal Features on features from Model 2 for Model 4
___

In [151]:
features_4= ['overall_qual_log', 'total_liv_sqft_log',
       'kitchen_qual', 'garage_total', 'bsmt_qual', 'year_built',
       'year_remod/add', 'fireplace_qu', 'totrms_abvgrd',
       'mas_vnr_area', 'full_bath']

In [152]:
X_4 = test_df_dummies_model_2[features_4]


In [153]:
poly = PolynomialFeatures(include_bias=False)

In [154]:
# Create X_poly
X_poly = poly.fit_transform(X_4)
X_poly.shape #numpy array. 

(879, 77)

In [155]:
X_poly_df = pd.DataFrame(X_poly, columns = poly.get_feature_names(features_4))

In [156]:
X_poly_df.to_csv(r"../data/X_poly_model_4.csv")

### 2.7 Adding Features for  Model 7
___

In [157]:
test_df_dummies_model_7 = test_df_dummies_model_2[['total_liv_sqft_log', 'overall_qual', 'total_liv_sqft',
       'overall_qual_log', 'exter_qual', 'kitchen_qual', 'garage_total',
       'bsmt_qual', 'year_built', 'year_remod/add', 'full_bath',
       'fireplace_qu', 'heating_qc', 'totrms_abvgrd','id']]

In [158]:
test_df_dummies_model_7.to_csv(r"../data/test_df_dummies_model_7.csv")

### 2.8 Adding Features for  Model 11
___

In [160]:
test_df_dummies_numeric = test_df_dummies.select_dtypes(include=[np.number])

In [164]:
add_neigborhoods = [, 'neighborhood_Landmrk', 'neighborhood_GrnHill', 'utilities_NoSeWa', 'heating_Wall']

In [166]:
add_neigborhoods

['heating_OthW',
 'neighborhood_Landmrk',
 'neighborhood_GrnHill',
 'utilities_NoSeWa',
 'heating_Wall']

In [168]:
test_df_dummies_numeric = pd.concat(
    [
        test_df_dummies_numeric,
        pd.DataFrame(
            [[0, 0,0,0,0]], 
            index=test_df_dummies_numeric.index, 
            columns=['heating_OthW',
 'neighborhood_Landmrk',
 'neighborhood_GrnHill',
 'utilities_NoSeWa',
 'heating_Wall']
        )
    ], axis=1
)



In [177]:
test_df_dummies_numeric.to_csv(r"../data/test_df_dummies_numeric.csv", index=False)


In [178]:
test_df_dummies_numeric.shape

(879, 104)